In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras import models

In [ ]:
actions = ['星期二', '幾月幾號', '台北', '星期一', '星期三', '有', '什麼', '完了嗎', '將近', '父母', '一共', '買', '家裡', '房子', '銀行', '昨天', '認識', '見她', '今天', '久', '比較', '一', '手語', '你', '他們', '星期五', '是嗎', '誰', '還沒有', '星期六', '桃園', '朋友', '高鐵到', '吃飯', '會不會呢', '我問你', '生日', '我們兩個', '租', '棒', '孩子', '零', '星期天', '明天', '運動', '星期四', '捷運站', '一樣', '上課', '我', '年齡', '無', '天氣', '相見', '名字']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
sequences = np.load('/content/drive/MyDrive/AI Python/Dataset/Refined_New_X_100.npy')
labels = np.load('/content/drive/MyDrive/AI Python/Dataset/Refined_New_Y_100.npy')

In [ ]:
np.array(sequences).shape 

In [ ]:
np.array(labels).shape

In [ ]:
X = np.array(sequences)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
y_test.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU, Bidirectional, BatchNormalization,ReLU,Activation
from tensorflow.keras.callbacks import TensorBoard


model = Sequential()
model.add(Bidirectional(GRU(165, return_sequences=True, activation='tanh'), input_shape=(60,165)))
model.add(Bidirectional(GRU(165, return_sequences=False, activation='tanh')))
model.add(Dense(64, activation='linear'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(np.array(actions).shape[0], activation='softmax'))
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_31 (Bidirectio (None, 60, 330)           328680    
_________________________________________________________________
bidirectional_32 (Bidirectio (None, 330)               492030    
_________________________________________________________________
dense_29 (Dense)             (None, 64)                21184     
_________________________________________________________________
batch_normalization_11 (Batc (None, 64)                256       
_________________________________________________________________
activation_8 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 55)                3575      
Total params: 845,725
Trainable params: 845,597
Non-trainable params: 128
_____________________________________________

In [ ]:
#AUTO_MODEL Architecture (100)
from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['categorical_accuracy'])
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
#checkpoint = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/AI Python/Dataset/Dropout.h5', monitor='val_categorical_accuracy', mode="max", save_best_only=True, verbose=1)
earlystopping = keras.callbacks.EarlyStopping(monitor='val_loss', mode="auto", patience=3, verbose=1)
rlr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1, mode='auto')

model.fit(X_train, y_train,validation_split = 0.1,
              epochs=30,
              batch_size = 32,
              callbacks=[rlr,earlystopping])


Epoch 1/30
147/147 [==============================] - 9s 29ms/step - loss: 2.8442 - categorical_accuracy: 0.3273 - val_loss: 2.0246 - val_categorical_accuracy: 0.5717
Epoch 2/30
147/147 [==============================] - 3s 21ms/step - loss: 1.3412 - categorical_accuracy: 0.7344 - val_loss: 0.9118 - val_categorical_accuracy: 0.7897
Epoch 3/30
147/147 [==============================] - 3s 21ms/step - loss: 0.6206 - categorical_accuracy: 0.8718 - val_loss: 0.4750 - val_categorical_accuracy: 0.8700
Epoch 4/30
147/147 [==============================] - 3s 21ms/step - loss: 0.3418 - categorical_accuracy: 0.9279 - val_loss: 0.3809 - val_categorical_accuracy: 0.8815
Epoch 5/30
147/147 [==============================] - 3s 22ms/step - loss: 0.2275 - categorical_accuracy: 0.9490 - val_loss: 0.2933 - val_categorical_accuracy: 0.9044
Epoch 6/30
147/147 [==============================] - 3s 21ms/step - loss: 0.1618 - categorical_accuracy: 0.9624 - val_loss: 0.2403 - val_categorical_accuracy: 0.944

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'test_loss={test_loss}')
print(f'test_accuracy={test_accuracy}')

9/9 [==============================] - 0s 11ms/step - loss: 0.0774 - categorical_accuracy: 0.9782
test_loss=0.07744494080543518
test_accuracy=0.9781818389892578
